# 数据读取及清洗

In [198]:
import pandas as pd
import numpy as np
import warnings#忽略打印的警告
warnings.filterwarnings('ignore')
df = pd.read_csv('meituan.csv')
df.head(1)

,web-scraper-order,web-scraper-start-url,店铺名称,详情页,详情页-href,店铺评分,人均消费,点评数,店铺地址,店铺电话,营业时间,配套服务,店铺菜系,点评标签,推荐菜1,推荐菜2,推荐菜3
0,1590852747-954,https://gz.meituan.com/meishi/pn1/,焱遇烧烤（金沙洲店）,焱遇烧烤（金沙洲店）3分2条评论白云区横沙大街214号1号铺人均￥60,https://www.meituan.com/meishi/1399342454/,3分,60.0,2.0,地址：白云区横沙大街214号1号铺,13560085375/13926422266,营业时间：周一至周日 17:00-03:00,提供wifi,广州烧烤烤串,NaN,烤羔羊肉串6串,烤亲亲五花肉,烤羔羊排串


In [199]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 965 entries, 0 to 964
Data columns (total 17 columns):
web-scraper-order        965 non-null object
web-scraper-start-url    965 non-null object
店铺名称                     965 non-null object
详情页                      965 non-null object
详情页-href                 965 non-null object
店铺评分                     913 non-null object
人均消费                     909 non-null float64
点评数                      906 non-null float64
店铺地址                     917 non-null object
店铺电话                     917 non-null object
营业时间                     910 non-null object
配套服务                     819 non-null object
店铺菜系                     917 non-null object
点评标签                     884 non-null object
推荐菜1                     917 non-null object
推荐菜2                     917 non-null object
推荐菜3                     917 non-null object
dtypes: float64(2), object(15)
memory usage: 128.3+ KB


In [200]:
df.dropna(axis=0, how='any', inplace=True)#删去有缺失值的行

In [201]:
#选取要进行分析的列
df = df[['店铺名称','店铺评分','人均消费','点评数',
         '店铺地址','店铺电话','营业时间','配套服务','店铺菜系','点评标签','推荐菜1','推荐菜2','推荐菜3']]

# 广州美食分布

In [202]:
from pyecharts import options as opts
from pyecharts.charts import Map
from pyecharts.faker import Faker

df['店铺区域'] = df['店铺地址'].str[3:6]#提取区域名
df['店铺区域'][df['店铺区域'] == '增城市'] = '增城区'
df['店铺区域'][df['店铺区域'] == '从化市'] = '从化区'
area = df['店铺区域'].value_counts()

x = area.index.tolist()
y = area.values.tolist()

c = (
    Map()
    .add(" ", [list(z) for z in zip(x,y)], "广州")
    .set_global_opts(
        title_opts=opts.TitleOpts(title="广州美食分布"), visualmap_opts=opts.VisualMapOpts()
    )
)
c.render('广州美食分布.html')
#店铺分布白云区,天河区,番禺区一带

'C:\\Users\\Administrator\\Desktop\\广州美食分布.html'

# 广州美食种类

In [203]:
from pyecharts import options as opts
from pyecharts.charts import Pie

df['菜系'] = df['店铺菜系'].str[2:]
cuisine = df['菜系'].value_counts()
cuisine= cuisine[cuisine>3]#选取数量靠前的菜系

x = cuisine.index.tolist()
y = cuisine.values.tolist()



c = (
    Pie()
    .add(
        "",
        [list(z) for z in zip(x,y)],
        radius=["40%", "75%"],
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(title="美食种类分布"),
        legend_opts=opts.LegendOpts(orient="vertical", pos_top="15%", pos_left="2%"),
    )
    .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}: {c}"))
)

c.render('美食种类分布.html')
#粤菜,火锅的数量最多

'C:\\Users\\Administrator\\Desktop\\美食种类分布.html'

# 店铺评分

In [204]:
from pyecharts import options as opts
from pyecharts.charts import Bar
from pyecharts.globals import ThemeType

df['店铺评分'] = df['店铺评分'].str.replace('分','').astype(float)#把分字去掉

#评分分级
df['评分分级'] = pd.cut(x=df['店铺评分'],bins =[0,0.5,1,1.5,2,2.5,3,3.5,4,4.5,6],
                   labels=['0-0.5','0.5-1','1-1.5','1.5-2','2-2.5','2.5-3','3-3.5','3.5-4','4-4.5','4.5-5'])
score = df['评分分级'].value_counts()
score = score[score>0]

x = score.index.tolist()
y = score.values.tolist()

c = (
    Bar(init_opts=opts.InitOpts(theme=ThemeType.LIGHT))
        .add_xaxis(x)
        .add_yaxis("", y)
        .set_series_opts(label_opts=opts.LabelOpts(position='top'))
        .set_global_opts(title_opts=opts.TitleOpts(title="店铺评分"))
    
)
c.render('店铺评分.html')
#3.5-4分的店铺最多

'C:\\Users\\Administrator\\Desktop\\店铺评分.html'

# 各区人均消费

In [205]:
from pyecharts import options as opts
from pyecharts.charts import Bar
from pyecharts.globals import ThemeType

mean_area = df.groupby('店铺区域')['人均消费'].mean().astype(int)#按区域划分人均消费


x = mean_area.index.tolist()
y = mean_area.values.tolist()

c = (
    Bar()
    .add_xaxis(x)
    .add_yaxis(" ", y)
    .set_global_opts(title_opts=opts.TitleOpts(title="各区人均消费"))
)
c.render('各区人均消费.html')
#天河区与海珠区人均消费最高

'C:\\Users\\Administrator\\Desktop\\各区人均消费.html'

# 白云区美食推荐

In [210]:
df_1 = df[df['店铺区域'] == '白云区']
recommend_baiyun = df_1.sort_values(by=['店铺评分','点评数','人均消费'],axis=0,ascending=False)#从评分,点评数量排序,选出排名靠前的美食店铺
recommend_baiyun.head()

,店铺名称,店铺评分,人均消费,点评数,店铺地址,店铺电话,营业时间,配套服务,店铺菜系,点评标签,推荐菜1,推荐菜2,推荐菜3,店铺区域,菜系,评分分级
275,芭蕉亚洲·泰国料理（同和金铂天地店）,5.0,65.0,8967.0,地址：白云区广州大道北同和大街1号金铂天地A座4楼（同和地铁D出口对面）,020-37414420,营业时间：周一至周日 11:30-21:30,提供wifi停车位,广州东南亚菜,上菜快(541)回头客(434)不用排队(59)停车免费(7)下午茶(132)朋友聚餐(30...,酸辣冬阴功汤,马来咖喱牛腩,菠萝炒饭,白云区,东南亚菜,4.5-5
888,湘食记地道湘菜,5.0,49.0,4621.0,地址：白云区广州大道北白灰场路8号（京溪南方医院c出口）,020-37356888,营业时间：周一至周日 11:00-21:00,提供wifi,广州川湘菜,菜品不错(295)上菜快(248)回头客(175)肉类好(111)交通便利(60)不用排队(...,铁铲鸡,爽口藕尖,老坛剁椒鱼头,白云区,川湘菜,4.5-5
163,同仁四季音乐主题餐厅（凯德广场云尚店）,5.0,75.0,3786.0,地址：白云区云城西路890号凯德广场B2层30/31号（探鱼对面）,020-86213306/020-86213260,营业时间：周一至周日 10:00-22:00,提供wifi,广州火锅,环境很好(404)味道赞(280)菜品不错(199)回头客(155)肉类好(112)交通便利...,椰子鸡,一级腊味煲仔饭,竹笙,白云区,火锅,4.5-5
104,佳鑫猪肚鸡（南方医院店）,5.0,53.0,3594.0,地址：白云区白灰场北街2号南方医院正大门对面（嘉裕太阳城外东方大药房煌上煌方向）,020-31523385/18026281720,营业时间：周一至周日 11:00-01:00,提供wifi停车位,广州火锅,回头客(171)上菜快(136)不用排队(28)停车方便(8)冬天吃过瘾(7)朋友聚餐(14...,胡椒猪肚鸡,腊味煲饭,猪肚鸡汤,白云区,火锅,4.5-5
752,川聚掌门,5.0,51.0,2820.0,地址：白云区广州大道北1420号圣地广场附楼3层（摩登百货旁边）,020-87200366,营业时间：周一至周日 10:00-02:00,提供wifi,广州川湘菜,回头客(324)上菜快(249)肉类好(57)海鲜棒(35)交通便利(14)不用排队(7)朋...,歪嘴辣掌,川江号子鱼,酸菜鱼,白云区,川湘菜,4.5-5
